PreProcessing step by step figure for report

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os
import cv2
import shutil
from PIL import Image
from skimage import exposure, restoration, color
from skimage.morphology import disk, closing
from skimage.filters import gaussian
from scipy.signal import convolve2d
import matplotlib.pyplot as plt
import glob

In [4]:
def preprocess_image(image_data):
    # preprocessing
    # image normalization: pixel values are in the range [0,1]
    normalized_data = image_data/np.max(image_data)
    plt.figure(figsize=(10,4))
    plt.subplot(1, 2, 1)
    plt.imshow(normalized_data, cmap='gray')
    plt.title('Normalized Image')
    plt.subplot(1, 2, 2)
    plt.hist(normalized_data.ravel(), bins=256, color='gray', alpha=0.7)
    plt.title('Histogram of Normalized Image')
    plt.tight_layout()
    plt.show()

    # convert image to greyscale
    gray_image_data = color.rgb2gray(normalized_data)
    plt.figure(figsize=(10,4))
    plt.subplot(1, 2, 1)
    plt.imshow(gray_image_data, cmap='gray')
    plt.title('Grayscale Image')
    plt.subplot(1, 2, 2)
    plt.hist(gray_image_data.ravel(), bins=256, color='gray', alpha=0.7)
    plt.title('Histogram of Grayscale Image')
    plt.tight_layout()
    plt.show()

    # image adjustment (stretch and adjust)
    # stretch the contrast of the image to represent the 5th and 95th percentiles of the pixel intensity distribution
    p1, p2 = np.percentile(gray_image_data, (5,95))
    stretched_data = exposure.rescale_intensity(gray_image_data, in_range=(p1, p2))
    plt.figure(figsize=(10,4))
    plt.subplot(1, 2, 1)
    plt.imshow(stretched_data, cmap='gray')
    plt.title('Stretched Image')
    plt.subplot(1, 2, 2)
    plt.hist(stretched_data.ravel(), bins=256, color='gray', alpha=0.7)
    plt.title('Histogram of Stretched Image')
    plt.tight_layout()
    plt.show()

    # Adjust the contrast and brightness of the image
    adjusted_data = exposure.adjust_gamma(stretched_data, gamma=0.5)
    plt.figure(figsize=(10,4))
    plt.subplot(1, 2, 1)
    plt.imshow(adjusted_data, cmap='gray')
    plt.title('Adjusted Image')
    plt.subplot(1, 2, 2)
    plt.hist(adjusted_data.ravel(), bins=256, color='gray', alpha=0.7)
    plt.title('Histogram of Adjusted Image')
    plt.tight_layout()
    plt.show()

    # image restoration (histogram equalization and weiner deconvolution)
    # histogram equalization
    histeq_data = exposure.equalize_hist(adjusted_data)
    plt.figure(figsize=(10,4))
    plt.subplot(1, 2, 1)
    plt.imshow(histeq_data, cmap='gray')
    plt.title('Histogram Equalized Image')
    plt.subplot(1, 2, 2)
    plt.hist(histeq_data.ravel(), bins=256, color='gray', alpha=0.7)
    plt.title('Histogram of Histogram Equalized Image')
    plt.tight_layout()
    plt.show()


    # weiner deconvolution
    # the psf used is a general approximation, a better estimate can be created for our data set (possible improvment, though probably small)
    psf = np.ones((3, 3)) / 9
    blurred_data = convolve2d(histeq_data, psf, mode='same', boundary='symm')
    plt.figure(figsize=(10,4))
    plt.subplot(1, 2, 1)
    plt.imshow(blurred_data, cmap='gray')
    plt.title('Blurred Image')
    plt.subplot(1, 2, 2)
    plt.hist(blurred_data.ravel(), bins=256, color='gray', alpha=0.7)
    plt.title('Histogram of Blurred Image')
    plt.tight_layout()
    plt.show()


    wiener_deconvolved_data = restoration.wiener(blurred_data, psf, balance=0.1)
    plt.figure(figsize=(10,4))
    plt.subplot(1, 2, 1)
    plt.imshow(wiener_deconvolved_data, cmap='gray')
    plt.title('Wiener Deconvolved Image')
    plt.subplot(1, 2, 2)
    plt.hist(wiener_deconvolved_data.ravel(), bins=256, color='gray', alpha=0.7)
    plt.title('Histogram of Wiener Deconvolved Image')
    plt.tight_layout()
    plt.show()

    # histogram equalization again
    histeq_data = exposure.equalize_hist(wiener_deconvolved_data)
    plt.figure(figsize=(10,4))
    plt.subplot(1, 2, 1)
    plt.imshow(histeq_data, cmap='gray')
    plt.title('Equalized Image (2nd time)')
    plt.subplot(1, 2, 2)
    plt.hist(histeq_data.ravel(), bins=256, color='gray', alpha=0.7)
    plt.title('Histogram Equalized Image (2nd time)')
    plt.tight_layout()
    plt.show()

    # adaptive histogram equalization
    adapeq_data = exposure.equalize_adapthist(histeq_data, clip_limit=0.02, kernel_size=None)
    plt.figure(figsize=(10,4))
    plt.subplot(1, 2, 1)
    plt.imshow(adapeq_data, cmap='gray')
    plt.title('Adaptive Histogram Equalized Image (1st time)')
    plt.subplot(1, 2, 2)
    plt.hist(adapeq_data.ravel(), bins=256, color='gray', alpha=0.7)
    plt.title('Histogram of Adaptive Histogram Equalized Image (1st time)')
    plt.tight_layout()
    plt.show()

    # adaptive histogram equalization again
    adapeq_data = exposure.equalize_adapthist(adapeq_data, clip_limit=0.99, kernel_size=None)
    plt.figure(figsize=(10,4))
    plt.subplot(1, 2, 1)
    plt.imshow(adapeq_data, cmap='gray')
    plt.title('Adaptive Histogram Equalized Image (2nd time)')
    plt.subplot(1, 2, 2)
    plt.hist(adapeq_data.ravel(), bins=256, color='gray', alpha=0.7)
    plt.title('Histogram of Adaptive Histogram Equalized Image (2nd time)')
    plt.tight_layout()
    plt.show()

    # Gaussian filtering 4 times
    filtered_image_data = adapeq_data.copy()
    for j in range(4):
        filtered_image_data = gaussian(filtered_image_data, sigma=1)
        filtered_image_data = restoration.denoise_tv_chambolle(filtered_image_data, weight=0.1)
        plt.figure(figsize=(10,4))
        plt.subplot(1, 2, 1)
        plt.imshow(filtered_image_data, cmap='gray')
        plt.title('Gaussian Filtered Image: Round ' +str(j+1))
        plt.subplot(1, 2, 2)
        plt.hist(filtered_image_data.ravel(), bins=256, color='gray', alpha=0.7)
        plt.title('Histogram of Gaussian Filtered Image: Round ' +str(j+1))
        plt.tight_layout()
        plt.show()

    # morphological closing operation
    selem = disk(5)
    closed_image_data = closing(filtered_image_data, selem)
    plt.figure(figsize=(10,4))
    plt.subplot(1, 2, 1)
    plt.imshow(closed_image_data, cmap='gray')
    plt.title('Morphologically Closed Image')
    plt.subplot(1, 2, 2)
    plt.hist(closed_image_data.ravel(), bins=256, color='gray', alpha=0.7)
    plt.title('Histogram of Morphologically Closed Image')
    plt.tight_layout()
    plt.show()


    return closed_image_data

In [7]:
#drive path to directory containg datasets
path = "..\Data Sets\Raw Datasets\Apoferritin\Apoferritin_001.png"

In [8]:
# Import image and convert to RGB values
image = cv2.imread(path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Ensure the image was loaded
if image is not None:
    # Display the original image and its histogram
    plt.figure(figsize=(10,4))
    plt.subplot(1, 2, 1)
    plt.imshow(image)
    plt.title('Original Image')
    plt.subplot(1, 2, 2)
    plt.hist(image.ravel(), bins=256, color='gray', alpha=0.7)
    plt.title('Histogram of Original Image')
    plt.tight_layout()
    plt.show()

    # Now call the preprocess_image function
    preprocessed_image = preprocess_image(image)
else:
    print("Image could not be loaded.")